In [ ]:
from src.kl_divergence import kl_divergence
from src.himmelblau import himmelblau
from src.stretch_move import stretch_move

from scipy.special import logsumexp
from scipy.stats import uniform

import numpy as np
import matplotlib.pyplot as plt
import corner
import emcee
import dynesty

In [ ]:
function = "himmelblau"
path = f"results/comparison/{function}"

file_name = f"{path}/{function}.txt"

In [ ]:
# Def log_likelihood function
def log_likelihood_himmelblau(x):
    return -himmelblau(x)

In [ ]:
# MCMC

# Initialize parameters
nwalkers = 40
ndim = 2
nsteps = 100000
a = 2
p0 = 8 * np.random.rand(nwalkers, ndim) - 4 # Initial grid

# Run MCMC
# Run the custom MCMC sampler
samples_mcmc, acceptance_count, delta_t_costum = stretch_move(log_likelihood_himmelblau, p0, nsteps, nwalkers, ndim, a)

# Calculate the mean acceptance rate
mean_acceptance_rate = acceptance_count / (nsteps * nwalkers)

# Calculate the autocorrelation times
autocorr_times = emcee.autocorr.integrated_time(samples_mcmc)

# Cut away 5 times the autocorellation time from samples
samples_mcmc = samples_mcmc[3 * int(np.max(autocorr_times)):]
samples_mcmc = samples_mcmc.reshape(-1, ndim)

# Print results
print("")
print("Costum implementation")
print("Autocorrelation times for each dimension:", autocorr_times)
print("Mean acceptance rate:", mean_acceptance_rate)
print(f"Runtime for {nsteps}:", delta_t_costum)

In [ ]:
# define the prior transform function
def prior_transform(uv):
    return 8.0 * uv - 4.0

# define the log-likelihood function
def log_likelihood(x):
    return -himmelblau(x)

ndim = 2

# define the number of living points used
n_live_points = 500

# define the sampler
sampler = dynesty.DynamicNestedSampler(log_likelihood, prior_transform, ndim, n_live_points)

# run the sampler
sampler.run_nested()

# get the results
results = sampler.results
results.summary()

samples_nested = results.samples


In [ ]:
kl_divergence(samples_nested, samples_mcmc, bins=50)

In [ ]:
# cut of the first 1000 samples from nested sampling
samples_nested = samples_nested[1000:]

In [ ]:
# Corner plot for mcmc
fig = corner.corner(samples_mcmc, labels=[r"$x$", r"$y$"], range=[(-5, 5), (-5, 5)])
fig.suptitle("MCMC samples")
fig.savefig(f"{path}/mcmc_corner_plot.pdf")
plt.show()

# Corner plot for nested sampling
fig = corner.corner(samples_nested, labels=[r"$x$", r"$y$"], range=[(-5, 5), (-5, 5)])
fig.suptitle("Nested sampling samples")
fig.savefig(f"{path}/nested_sampling_corner_plot.pdf")
plt.show()

In [ ]:
# Plot both corner plots in the same figure
fig = corner.corner(samples_mcmc, labels=[r"$x$", r"$y$"], range=[(-5, 5), (-5, 5)], color="blue", hist_kwargs={"density": True})
fig = corner.corner(samples_nested, labels=[r"$x$", r"$y$"], range=[(-5, 5), (-5, 5)], color="red", fig=fig, hist_kwargs={"density": True})
fig.suptitle("MCMC and Nested sampling samples")
fig.savefig(f"{path}/{function}_comparison.pdf")
plt.show()